In [71]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.agent_toolkits.load_tools import load_tools
import os
from langchain.tools import tool
from langchain.vectorstores import FAISS
# 设置LLM
from langchain_community.chat_models import ChatTongyi
# 引入上下文压缩
from langchain.retrievers import ContextualCompressionRetriever # 上下文压缩检索
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.react.agent import create_react_agent
from langchain import hub
from langchain_community.chat_models.tongyi import ChatTongyi
import os
from langchain.tools import tool
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import AgentExecutor
from langchain_core.language_models.base import BaseLanguageModel
import time
from typing import (
    Any,
    Dict,
    List,
    Optional,
    Tuple,
)
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.callbacks import CallbackManagerForChainRun
from langchain_core.language_models import BaseLanguageModel
from langchain_core.utils.input import get_color_mapping
from langchain.prompts import PromptTemplate



In [72]:
# 初始化Tongyi模型
api_key = os.getenv("KEY_TONGYI")
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0
)

In [73]:
# 重写了AgentExecutor
class LJHAgentExecutor(AgentExecutor):
  
  llm: BaseLanguageModel
  
  def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """Run text through and get agent response."""
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.tools], excluded_colors=["green", "red"]
        )
        intermediate_steps: List[Tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # We now enter the agent loop (until it returns something).
        while self._should_continue(iterations, time_elapsed):
            next_step_output = self._take_next_step(
                name_to_tool_map,
                color_mapping,
                inputs,
                intermediate_steps,
                run_manager=run_manager,
            )
            # print("刚刚进入循环:", next_step_output)
            # print("inputs:", inputs)
            
            if isinstance(next_step_output, AgentFinish):
                # print('if isinstance(next_step_output, AgentFinish):', next_step_output)
                return self._return(
                    next_step_output, intermediate_steps, run_manager=run_manager
                )

            intermediate_steps.extend(next_step_output)
            if len(next_step_output) == 1:
                next_step_action = next_step_output[0]
                # See if tool should return directly
                # print("if len(next_step_output) == 1:",next_step_action)
                tool_return = self._get_tool_return(next_step_action)
                # print("22222222222222222222222", tool_return)
                
                if tool_return is not None:
                    return self._return(
                        tool_return, intermediate_steps, run_manager=run_manager
                    )
            
            iterations += 1
            time_elapsed = time.time() - start_time
        # 改了
        # output = self.agent.return_stopped_response(
        #     self.early_stopping_method, intermediate_steps, **inputs
        # )
        
        # print("最后输出的内容:", output)
        ai_response = self.llm.invoke(inputs['input'])
        # print("ai_response",ai_response.content)
        # output['output'] = ai_response.content
        final_output = AgentFinish(
            return_values={"output": ai_response.content},
            log=''
        )
        # print(output)
        # print(final_output)
        return self._return(final_output, intermediate_steps, run_manager=run_manager)



In [82]:
class ChatDoc:
    def __init__(self, faiss_db_path, system):
        self.faiss_db_path = faiss_db_path
        self.template = [
            ("system",
            "你是一个精通C语言的C语言专家，在这个阶段你的主要目标是根据输入的内容匹配MISRA C2012标准的相关信息，输入内容{question}通常包含着违反的具体规则和错误代码,{context}通常包含着最相近的MISRA C2012细则，你需要根据输入的规则查阅MISRA C2012规范, 确定违反的规范细则是否准确，并将准确的细则和输入的代码一并返回"),
            ("human", "我会给你部分C语言代码和分析结果以及我认为它具体违反了MISRA C2012的哪些规范，你来帮我检查并纠正\n"),
            ("ai", "没问题!"),
            ("human", "{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

    # 加载向量数据库
    def load_vector_db(self):
        # 加载已经保存的 FAISS 索引
        hf = HuggingFaceEmbeddings(
            model_name="E:\\AAAAWork\\python\\models\\EMB\\bce-embedding-base_v1",  # 替换为实际的模型路径
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
        db = FAISS.load_local(self.faiss_db_path, hf, allow_dangerous_deserialization=True)
        return db

    # 提问并找到相关文本块
    def askAndFindFiles(self, question):
        db = self.load_vector_db()  # 加载已有的向量数据库
        print("db:",db)
        # 采用上下文压缩的方式
        retriever = db.as_retriever()
        compressor = LLMChainExtractor.from_llm(llm=llm)
        compressor_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )
        return compressor_retriever.get_relevant_documents(query=question)

    # 用自然语言和文档聊天
    def chatWithDoc(self, question):
        _context = ""
        context = self.askAndFindFiles(question)
        for i in context:
            _context += i.page_content
        message = self.prompt.format_messages(context=_context, question=question)
        print("chatWithDoc:",message)
        return llm.invoke(message)


In [83]:
@tool
def content_info_tool(query: str) -> str:
    """
    当需要了解MISRA C2012标准的相关信息，你可以调用此方法，
    你调用此工具需要输入信息必须包含违反的MISRA C 2012规则编码 , 违规内容的文字描述 以及 被检测的代码，
    文字描述应该包含你想到的可能违反的MISRA C 2012的规范细则，
    被检测代码应包含上下至少各三行代码以及相关变量的定义，而不是只有一行，
    规则编码 ， 违规内容的文字描述 ， 被检测的代码 三者缺一不可！
    然后返回MISRA C2012中对应的规范内容！
    """
    print("调用了自定的tool",query)
    query += "它违背了预制文档向量化数据库里面提到的哪一点？中文回答"
    content = contents_search(query)
    
    return content
      
# content_info_tool工具会使用的函数
# 读取以及存储的文本向量回答问题
def contents_search(question):
  contents_chat = ChatDoc("./contents.index", "你是一个查看被提供的C语言是不是符合C2012标准的秘书。如果不符合标准，你会返回是不符合你参考文本中的哪一条标准,只需要返回对应的标号就行。你会根据下面提供的上下文内容来继续回答问题.")
  response = contents_chat.chatWithDoc(question)
  
  return response.content




In [84]:
tools = load_tools(["llm-math"], llm)  

# 添加自定义工具到工具列表中
tools.append(content_info_tool)

In [93]:
hub_api_key = os.getenv("KEY_SMITH")

# prompt = hub.pull("ljh/prompt_create_react_agent", api_key=hub_api_key)
# Prompt 模板
template = '''尽力用中文回答下面的问题，你可以使用下面的工具:

{tools}

使用下面的格式:

Question: 你必须回答的问题
Thought: 你应该经常考虑该做什么，如果使用工具无法获得准确的结果，你应该终止操作，返回"陛下，臣妾做不到啊"这句话
Action: 要采取的行动，应该是[{tool_names}]中的一个
Action Input: 动作的输入
Observation: 行动的结果
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: 我知道最终的答案了
Final Answer: 原始输入问题的最终答案

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(
  llm=llm,
  tools=tools,
  prompt=prompt,
)
agent_executor = LJHAgentExecutor(
  llm=llm,
  agent=agent, 
  tools=tools, 
  handle_parsing_errors=True,
  max_iterations=3 # 最多迭代3次
)

In [96]:
question = """
#include <stdio.h>

int main() {
    /* 这是一个注释开始
    /* 这是嵌套的注释 */
    printf("This is a test.\n");
    return 0;
}


"""

s=agent_executor._call({"input": question})


In [95]:
s

{'output': '给定的C语言代码片段是一个简单的程序，包含了一些基本的C语言元素：\n\n```c\n#include <stdio.h>\n```\n\n这行代码包含了`stdio.h`头文件，它是用于标准输入输出的库，提供了像`printf`这样的函数。\n\n```c\nint main() {\n    /* 这是一个注释开始\n    /* 这是嵌套的注释 */\n    printf("This is a test.\\n");\n    return 0;\n}\n```\n\n- `int main()`：这是主函数的声明，所有C程序的执行都是从这里开始的。\n- `printf("This is a test.\\n");`：这是一个使用`printf`函数的例子，用于输出字符串"This is a test."到控制台，并且在字符串末尾添加了一个换行符`\\n`。\n- 注释：代码中使用了两种注释形式。单行注释以`/*`开始并以`*/`结束，而多行注释则使用`/*`开始和`*/`结束。多行注释内部可以嵌套，即一个注释块内可以包含另一个注释块。\n\n总的来说，这段代码遵循了C语言的基本语法和编程实践，但是否符合MISRA C 2012标准取决于具体的规则集，特别是与内存管理、指针使用、错误处理等相关的规则。如果没有具体的违规规则，那么这段代码在标准的C语言实践中是正确的。'}